In [349]:
# Converted by Tim Stols
# python Explainer_Experiments.py --model_name CMPNN \
#                                                        --attribution_name GradInput \
#                                                        --data_path ../MolRep/Datasets/Metabolism/admet2.1_rlm_merge.csv \
#                                                        --dataset_name RLM \
#                                                        --smiles_col COMPOUND_SMILES \
#                                                        --target_col CLF_LABEL \
#                                                        --task_type Multi-Classification \
#                                                        --multiclass_num_classes 3 \
#                                                        --output_dir ../Outputs

In [350]:
import copy
from datetime import datetime

import numpy as np

In [351]:
# OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# # OUTPUT_DIR = '../Outputs'
# DATASET = '3MR'
# 
# # GNN
# MODEL = 'GraphNet' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']
# 
# # explainer
# ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline
# 
# DATAPATH = '../DataSets/3MR/toy_label_mw350.csv'
# SMILESCOL = 'SMILES'
# TARGETCOL = 'label_full'
# 
# ATTRIBUTIONPATH = '../DataSets/3MR/attributions.npz'
# 
# TASKTYPE = 'Regression' # Can be 'Multi-Classification', 'Classification', 'Regression'
# MULTICLASS_NUM_CLASSES = 2 # Can be 3
# TESTING = True
# SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [352]:
# OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# # OUTPUT_DIR = '../Outputs'
# DATASET = 'Liver'
# 
# # GNN
# MODEL = 'CMPNN' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']
# 
# # explainer
# ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline
# 
# DATAPATH = '../Datasets/XAI/Liver/liver.csv'
# SMILESCOL = 'SMILES'
# TARGETCOL = 'label'
# 
# ATTRIBUTIONPATH = '../Datasets/XAI/Liver/attributions.npz'
# 
# TASKTYPE = 'Classification' # Can be 'Multi-Classification', 'Classification', 'Regression'
# MULTICLASS_NUM_CLASSES = 2 # Can be 3
# TESTING = True
# SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [353]:
from datetime import datetime
OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# OUTPUT_DIR = '../Outputs'
DATASET = 'Benzene'

# GNN
MODEL = 'CMPNN' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']

# explainer
ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline

DATAPATH = '../Datasets/XAI/Benzene/benzene_smiles.csv'
# DATAPATH = '../Datasets/XAI/Benzene/benzene_smiles - with benzene.csv'
SMILESCOL = 'SMILES'
TARGETCOL = 'label'

ATTRIBUTIONPATH = '../Datasets/XAI/Benzene/attributions.npz'

TASKTYPE = 'Classification' # Can be 'Multi-Classification', 'Classification', 'Regression'
MULTICLASS_NUM_CLASSES = 2 # Can be 3
TESTING = True
SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [354]:
import rdkit.Chem
from MolRep.Utils.logger import Logger
from MolRep.Explainer.explainerExperiments import ExplainerExperiments
from MolRep.Explainer.explainerDataWrapper import ExplainerDatasetWrapper
from MolRep.Utils.config_from_dict import Grid, Config, DatasetConfig
from pathlib import Path
import os
from numba.cuda import args

LOGGER_BASE = os.path.join(OUTPUT_DIR, "Logger", f"{DATASET}_explainer")
logger = Logger(str(os.path.join(LOGGER_BASE, f"{MODEL}_{DATASET}_explainer_by_{ATTRIBUTION}.log")), mode='a')



# output for vector groups
data_dir = Path('../MolRep/Data')
split_dir = Path('../MolRep/Splits')
os.makedirs(data_dir, exist_ok=True)
os.makedirs(split_dir, exist_ok=True)

# moving down to have folders for different epoch numbers
# svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}")
# 
# # model output
# output_dir = Path(OUTPUT_DIR)
# MODELPATH = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", f"{MODEL}.pt")
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# os.makedirs(svg_dir, exist_ok=True)

# torch.set_num_threads(1)")
data_stats = {
            'name': DATASET,
            'path': DATAPATH,
            'smiles_column': SMILESCOL,
            'target_columns': [TARGETCOL],
            'attribution_path': ATTRIBUTIONPATH,
            'task_type': TASKTYPE,
            'multiclass_num_classes': MULTICLASS_NUM_CLASSES,
            'metric_type': 'rmse' if TASKTYPE == 'Regression' else ['acc', 'auc', 'f1', 'precision', 'recall'],
            'split_type': SPLITTYPE
}

if TESTING:
    data_stats['additional_info'] = {"splits":'SPLIT'}

In [355]:
config_file = '../MolRep/Configs/config_{}.yml'.format(MODEL)
model_configurations = Grid(config_file)
model_configuration = Config(**model_configurations[0])
dataset_configuration = DatasetConfig(DATASET, data_dict=data_stats)

CMPNN
{'GIN': <class 'MolRep.Models.graph_based.GIN.GIN'>, 'ECC': <class 'MolRep.Models.graph_based.ECC.ECC'>, 'DGCNN': <class 'MolRep.Models.graph_based.DGCNN.DGCNN'>, 'DiffPool': <class 'MolRep.Models.graph_based.DiffPool.DiffPool'>, 'GraphSAGE': <class 'MolRep.Models.graph_based.GraphSAGE.GraphSAGE'>, 'GAT': <class 'MolRep.Models.graph_based.GAT.GAT'>, 'GraphNet': <class 'MolRep.Models.graph_based.GraphNet.GraphNet'>, 'MPNN': <class 'MolRep.Models.graph_based.MPNN.MPNN'>, 'CMPNN': <class 'MolRep.Models.graph_based.CMPNN.CMPNN'>, 'DMPNN': <class 'MolRep.Models.graph_based.DMPNN.DMPNN'>, 'MAT': <class 'MolRep.Models.sequence_based.MAT.MAT'>, 'CoMPT': <class 'MolRep.Models.sequence_based.CoMPT.CoMPT'>, 'BiLSTM': <class 'MolRep.Models.sequence_based.BiLSTM.BiLSTM'>, 'SALSTM': <class 'MolRep.Models.sequence_based.SALSTM.SALSTM'>, 'Transformer': <class 'MolRep.Models.sequence_based.Transformer.Transformer'>, 'VAE': <class 'MolRep.Models.unsupervised_based.VAE.VAE'>, 'RandomForest': <class

In [356]:
###### graphics go here
svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{model_configuration['num_epochs']}_epochs")

# model output
output_dir = Path(OUTPUT_DIR)
MODELPATH = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", f"{MODEL}.pt")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(svg_dir, exist_ok=True)

In [357]:
exp_path = os.path.join(output_dir, f'{model_configuration.exp_name}_{dataset_configuration.exp_name}_explainer')

dataset = ExplainerDatasetWrapper(dataset_config=dataset_configuration,
                                  model_name=model_configuration.exp_name,
                                  split_dir=split_dir, features_dir=data_dir)


In [358]:

explainer_experiment = ExplainerExperiments(model_configuration, dataset_configuration, exp_path)


# test_smiles = dataset.get_smiles_list(testing=True)
# print("test")
# print(test_smiles[0:3])
# 
# train_smiles = dataset.get_smiles_list(testing=False)
# print("train")
# # benzene added to train set despite assignment to testing?
# print(train_smiles[0:3])
# 

# validation, with new data breaks
explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})
# if not os.path.exists(MODELPATH):
#     explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})

IndexError: index 12000 is out of bounds for axis 0 with size 12000

In [ ]:
results, atom_importance, bond_importance = explainer_experiment.molecule_importance(dataset=dataset, attribution=ATTRIBUTION, logger=logger, other={'model_path':MODELPATH}, testing=TESTING)

# model + dataset + explainer \   = is smiles representations of the data? 
# .\Outputs\CMPNN_Liver_explainer\CMPNN_explained_by_IG_oof.csv

In [ ]:
import pandas as pd

import rdkit.Chem as chem

id = chem.MolFromSmiles("c1ccccc1")
id2 = chem.MolFromSmiles("C1CCCCC1")

print(id, id2)

In [ ]:

# logger.log('Test results: %s' % str(results))

# print(attribution_results)

if DATASET in ['hERG', 'CYP3A4']:
    attribution_results, opt_threshold = explainer_experiment.evaluate_cliffs(dataset, atom_importance, bond_importance)
else:
    binary = True if ATTRIBUTION == 'MCTS' else False
    #TODO not working, because ATTRIBUTIONPATH is not set and attributions are not saved
    attribution_results, opt_threshold = explainer_experiment.evaluate_attributions(dataset, atom_importance, bond_importance, binary=binary)

In [ ]:

#TODO not working
# logger.log('attribution_results:' + str(attribution_results))
# logger.log('opt_threshold:' + str(opt_threshold))

# create visualisations using  the dataset and learned atom and bond importances

 
# c1cc(ncc1NC(=O)C1CCCCC1)n1cc(cn1)S(=O)(=O)N 

# def get_smiles_list(self, testing=True, training=False):
#     whole_smiles = self.whole_data_df.loc[:,self.smiles_col].values
#     if testing:
#         testset_indices = self.splits[0]["test"]
#     elif training:
#         testset_indices = self.splits[0]["model_selection"][0]["train"]
#     else:
#         testset_indices = np.arange(len(whole_smiles))
#     return whole_smiles[testset_indices]


# testing = True returns both benzene and the carbon hex ring, b = 1 test, c6 = 0 train, 3rd = 0 train

# test_smiles = dataset.get_smiles_list(testing=True)
# print("test")
# print(test_smiles[0:3])
# 
# train_smiles = dataset.get_smiles_list(testing=False)
# print("train")
# # benzene added to train set despite assignment to testing?
# print(train_smiles[0:3])

In [ ]:

explainer_experiment.visualization(dataset, atom_importance, bond_importance, svg_dir=svg_dir, testing=TESTING)

df = pd.DataFrame(
    {'SMILES': dataset.get_smiles_list(), 'Atom_importance': atom_importance, 'Bond_importance':bond_importance}
)
df.to_csv(os.path.join(svg_dir, "importances.csv"), index=False)

In [ ]:
data = pd.read_csv(DATAPATH)[['SPLIT', 'label']]

import pprint

# print("Epochs: ", model_configuration['num_epochs'])
pp = pprint.PrettyPrinter(indent=1)
print(model_configuration['num_epochs'])
print(results['acc'])
print(results['auc'])
print(results['f1'])
print(results['precision'])
print(results['recall'])
print()
print(attribution_results['Attribution AUROC'])
print(attribution_results['Attribution F1'])
print(attribution_results['Attribution ACC'])
print(attribution_results['Attribution Precision'])
print(attribution_results['Attribution AUROC Mean'])
print(attribution_results['Attribution ACC Mean'])

In [395]:
import copy

def split_molecule(mol_idx:int, threshold=1e-4):
    """Splits the given molecule into explaining and non-explaining SMILES, returns those and the SMILES of the molecule. Threshold value is the same as in explainerExperiments.py visualization()"""
    smile = df.SMILES.iat[mol_idx]

    # TODO get test indices, index by the index in the test set
    importances = df.Atom_importance.iat[mol_idx]

    # atom importance with big set
    
    original_mol = rdkit.Chem.RWMol(rdkit.Chem.MolFromSmiles(smile))
     
    non_explaining_atom_idx = []
    explaining_atom_idx = []
    for idx in range(len(original_mol.GetAtoms())):
        if importances[idx] < threshold:
            non_explaining_atom_idx.append(idx)
        else:
            explaining_atom_idx.append(idx)
            
    # reverse because removing atoms causes indices of others to shift
    non_explaining_atom_idx.reverse()
    explaining_atom_idx.reverse()
    
    # find (non)explaining bonds where at least one end is a non-explaining atom
    non_explaining_bonds = []
    explaining_bonds = []
    
    for bond in original_mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        
        # if both end and start in explaining atoms, the edge is explaining 
        if (i and j) not in non_explaining_atom_idx :
            explaining_bonds.append((i, j))
            explaining_bonds.append((j, i))
        else:
            non_explaining_bonds.append((i, j))
            non_explaining_bonds.append((j, i))
            
            # remove i or j from explaining atoms to keep the bond in non-explaining molecules
            if i in explaining_atom_idx: explaining_atom_idx.remove(i)
            if j in explaining_atom_idx: explaining_atom_idx.remove(j)
    
    # explaining molecule has non-explaining bonds and atoms removed
    explaining_mol = copy.deepcopy(original_mol)
    for (i, j) in non_explaining_bonds:
        explaining_mol.RemoveBond(i, j)
    
    for idx in non_explaining_atom_idx:
        explaining_mol.RemoveAtom(idx)

    explaining_smile = ""
    if len(explaining_mol.GetAtoms() > 0):
        explaining_smile = rdkit.Chem.MolToSmiles(explaining_mol)
    
    # comprehensiveness uses non-explaining atoms and edges
    non_explaining_mol = copy.deepcopy(original_mol)
    
    for (i, j) in explaining_bonds:
        non_explaining_mol.RemoveBond(i, j)
    
    # remove explaining atoms unless it has a bond to a non-explaining atom    
    for i in explaining_atom_idx:
        non_explaining_mol.RemoveAtom(i)
    
    non_explaining_smile = ""
    # edge case: there are no non-explaining edges
    if len(non_explaining_mol.GetAtoms()) > 0:
        non_explaining_smile = rdkit.Chem.MolToSmiles(non_explaining_mol)
    
    # if (mol_idx == 3):
    #     print("hiii")
    #     print(smile)
    #     print(explaining_smile)
    #     print(non_explaining_smile)
    #     # print(importances)
    # print(smile)
    
    return smile, explaining_smile, non_explaining_smile

In [ ]:
import torch
from MolRep.Featurization.MPNN_embeddings import MolGraph, BatchMolGraph
from rdkit.Chem import MolFromSmiles, rdmolops

def tensors_to_device(smile, batch):
    """Sets nodes, edges, a2b, b2a and adjacency matrix tensors of a molecule to the device"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    adj = rdmolops.GetAdjacencyMatrix(MolFromSmiles(smile))
    
    list = {
        'nodes' : batch.f_atoms, 
        'edges' : batch.f_bonds,
        'a2b' : batch.a2b,
        'b2a' : batch.b2a,
        'adjacency' : torch.FloatTensor(adj) 
    }
    
    for i in list:
        list[i].to(device)
        
    return list['nodes'], list['edges'], list['a2b'], list['b2a'], list['adjacency']

In [424]:
import re

def clean_smiles(s:str) -> str:
    """Takes a (fragmented) SMILES as input and returns valid fragments of it as a new SMILES. Does nothing if input is already valid.
    e.g: C.CC.F.O.n.n -> C.CC.F.O
    """
    cs = ""
    # 1. split string
    subm = s.split('.')

    # 2. if piece converts to a molgraph, append to cs
    for (i, sm) in enumerate(subm):
        try:
            MolGraph(sm)
            cs += sm
            if i < len(subm) - 1:
                cs += '.'
        except:
            if i < len(subm) - 1:
                cs += '.'
            continue

    # 3. remove repeating . and the last . from the string
    cs = re.sub("\.+", ".", cs)
    # ReGeX from https://stackoverflow.com/a/3331982
    cs = re.sub("\.([^.]*)$", "", cs)
    return cs

FileNotFoundError: [Errno 2] No such file or directory: '../Outputs/20240528_115402\\CMPNN_Benzene_explainer\\CMPNN.pt'

In [ ]:
def comp_and_suff(i) :
    """Calculate comprehensiveness and sufficiency for a molecule with index i."""
    model = explainer_experiment.get_model(dataset, other={'model_path':MODELPATH})
    model.eval() # set model to evaluation mode

    smile, explaining_smile, non_explaining_smile = split_molecule(i)

    preds = []
    for s in [smile, explaining_smile, non_explaining_smile] :
        # split may have resulted in invalid smiles
        cs = clean_smiles(s)

        print("s", s)
        print("cs", cs)

        # 3. try to convert to molgraph
        # TODO if iterating explaining smile -> do each fragment separately
        try:
            mol = MolGraph(cs)
            g_input = BatchMolGraph( [mol])
            atoms, bonds, a2b, b2a, adjacency = tensors_to_device(cs, g_input)

            with torch.no_grad():
                # TODO investigate None params
                #   Args for mol2graph:
                # - mols: A list of SMILES or a list of RDKit molecules.
                # - atom_descriptors_batch: A list of 2D numpy array containing additional atom descriptors to featurize the molecule
                pred = model([[cs], None, None]).item()
                # print(pred)

            print(i, ' pred for ', cs, ': ', pred)
            preds.append(pred)
        except:
            # in case smile is an empty string, append None to indicate comp or suff can't be calculated
            preds.append(None)
            continue

    g_pred = preds[0] # prediction of original molecule
    e_pred = preds[1] # prediction of explaining molecule
    n_pred = preds[2] # prediction of non-explaining molecule

    # f = model prediction
    # comp = f(smile) - f(non_explaining_smile)
    # suff = f (smile) - f(explaining_smile)

    comp = None
    suff = None

    if n_pred is not None:
        comp = g_pred - n_pred
    if e_pred is not None:
        suff = g_pred - e_pred

    return [g_pred, comp, suff]

In [ ]:
def comp_and_suff_data(df):
    c_and_s = []
    # TODO 1 breaks at idx 3
    # test or train indices

    for i in range(len(df)):
        # only calculate comp and suff for molecules in the test set i.e. which have explanations
        if i in dataset.get_smiles_idxs(testing=True):
            vals = comp_and_suff(i)
            print(i, vals)
            c_and_s.append(vals)
    return c_and_s

g_pred, comp, suff = comp_and_suff(3)
# print('comp:', comp)
# print('suff:', suff)

c = comp_and_suff_data(df)
print(type(c))

In [422]:
# TODO 2 further work
# save results of comp-suff for transparency
# use a different explainer on the model
# save those results

cs C.CC.F.O..
cs C.CC.F.O
